In [10]:
import pandas as pd
import pymongo
from splinter import Browser 
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [11]:
#
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/fraolabebe/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


### NASA Mars News

* Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [18]:
def mars_news():
    browser.visit('https://mars.nasa.gov/news/')
    title = browser.find_by_css('div.content_title a').text
    paragraph = browser.find_by_css('div.article_teaser_body').text
    
    return title, paragraph


### JPL Mars Space Images - Featured Image

* Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).

* Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.

* Make sure to find the image url to the full size `.jpg` image.

* Make sure to save a complete url string for this image.

In [24]:
def featured_image:
    
    browser.visit('https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars')
    browser.find_by_id('full_image').click()
    browser.links.find_by_partial_text('more info').click()
    featured_image_url = browser.find_by_css('figure.lede a')['href']
    
    return featured_image_url


### Mars Facts

* Visit the Mars Facts webpage [here](https://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

* Use Pandas to convert the data to a HTML table string.


In [25]:
def mars_facts():
    mars_facts_html = pd.read_html('https://space-facts.com/mars/')[0]\
    .rename(columns={0:'labels',1:'measurements'})\
    .to_html(classes='table table-stripped')
    return mars_facts_html
    





### Mars Hemispheres

* Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.

* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.

* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [27]:
def hemisphere_img():
    img_list = []
    browser.visit('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')
    img_urls = browser.find_by_css('a.product-item h3')

    print(img_urls)

    for link in range(len(img_urls)):
        hems_imgs = {}
        browser.find_by_css('a.product-item h3')[link].click()
        img_url = browser.links.find_by_text("Sample")['href']
        img_title = browser.find_by_css('h2.title').text
        img_list.append({'title': img_title, 'img_url': img_url})
        browser.back()
        
        return img_list

    print(img_list)
    
    

## Insert into MongoDB

Use MongoDB with Flask templating to create a new HTML page that displays all of the information that was scraped from the URLs above.

* Start by converting your Jupyter notebook into a Python script called `scrape_mars.py` with a function called `scrape` that will execute all of your scraping code from above and return one Python dictionary containing all of the scraped data.

* Next, create a route called `/scrape` that will import your `scrape_mars.py` script and call your `scrape` function.

  * Store the return value in Mongo as a Python dictionary.

* Create a root route `/` that will query your Mongo database and pass the mars data into an HTML template to display the data.

* Create a template HTML file called `index.html` that will take the mars data dictionary and display all of the data in the appropriate HTML elements. Use the following as a guide for what the final product should look like, but feel free to create your own design.

In [49]:
# scrape all
def scrape_all():
    
    title, paragraph = mars_news()
    featured_image_url = featured_image
    mars_facts_html = mars_facts
    img_list = hemisphere_img()

    nasa_document = {
        'news_title': title,
        'news_paragraph': paragraph,
        'featured_image': featured_image,
        'mars_facts_htmt': mars_facts
    }
    
    return nasa_document
    print(nasa_document)

In [50]:
# Connect to MongoDB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Connect to mars_app database
db = client.mars_app

# Connect to mars collection
mars = db.mars

#gather document to insert

data_document = scrape_all()
mars.insert_one(data_document)

